Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import datetime


Load the datasets

In [2]:
df_customers = pd.read_csv('/Users/eranola/Documents/projects/sales_analysis/data/raw/olist_customers_dataset.csv')
df_geolocation = pd.read_csv('/Users/eranola/Documents/projects/sales_analysis/data/raw/olist_geolocation_dataset.csv')
df_order_items = pd.read_csv('/Users/eranola/Documents/projects/sales_analysis/data/raw/olist_order_items_dataset.csv')
df_order_payments = pd.read_csv('/Users/eranola/Documents/projects/sales_analysis/data/raw/olist_order_payments_dataset.csv')
df_order_reviews = pd.read_csv('/Users/eranola/Documents/projects/sales_analysis/data/raw/olist_order_reviews_dataset.csv')
df_orders = pd.read_csv('/Users/eranola/Documents/projects/sales_analysis/data/raw/olist_orders_dataset.csv')
df_products = pd.read_csv('/Users/eranola/Documents/projects/sales_analysis/data/raw/olist_products_dataset.csv')
df_sellers = pd.read_csv('/Users/eranola/Documents/projects/sales_analysis/data/raw/olist_sellers_dataset.csv')
df_product_category_translation = pd.read_csv('/Users/eranola/Documents/projects/sales_analysis/data/raw/product_category_name_translation.csv')

In [4]:
# Convert relevant columns in the df_orders dataset to datetime
df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'])
df_orders['order_approved_at'] = pd.to_datetime(df_orders['order_approved_at'])
df_orders['order_delivered_carrier_date'] = pd.to_datetime(df_orders['order_delivered_carrier_date'])
df_orders['order_delivered_customer_date'] = pd.to_datetime(df_orders['order_delivered_customer_date'])
df_orders['order_estimated_delivery_date'] = pd.to_datetime(df_orders['order_estimated_delivery_date'])

# Convert relevant columns in the df_order_reviews dataset to datetime
df_order_reviews['review_creation_date'] = pd.to_datetime(df_order_reviews['review_creation_date'])
df_order_reviews['review_answer_timestamp'] = pd.to_datetime(df_order_reviews['review_answer_timestamp'])

# Convert relevant columns in the df_order_items dataset to datetime
df_order_items['shipping_limit_date'] = pd.to_datetime(df_order_items['shipping_limit_date'])


In [5]:
# Merge datasets
order_details = pd.merge(df_order_items, df_order_payments, on='order_id')
order_details = pd.merge(order_details, df_orders, on='order_id')

# Calculate total revenue per order
order_details['total_revenue'] = order_details['price'] + order_details['payment_value']

# Calculate total revenue per customer
customer_revenue = order_details.groupby('customer_id')['total_revenue'].sum()

# Calculate average revenue per customer (LTV)
ltv = customer_revenue.mean()

print(f"Lifetime Value (LTV): {ltv}")

Lifetime Value (LTV): 349.8429032585009


In [6]:
# Convert the order purchase timestamp to datetime
df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'])

# Get the first purchase date per customer
first_purchase_date = df_orders.groupby('customer_id')['order_purchase_timestamp'].min().reset_index()
first_purchase_date.columns = ['customer_id', 'first_purchase_date']

# Extract year and month from the first purchase date
first_purchase_date['year_month'] = first_purchase_date['first_purchase_date'].dt.to_period('M')

# Count new users per month
new_users_monthly = first_purchase_date['year_month'].value_counts().sort_index()

In [7]:
new_users_monthly

year_month
2016-09       4
2016-10     324
2016-12       1
2017-01     800
2017-02    1780
2017-03    2682
2017-04    2404
2017-05    3700
2017-06    3245
2017-07    4026
2017-08    4331
2017-09    4285
2017-10    4631
2017-11    7544
2017-12    5673
2018-01    7269
2018-02    6728
2018-03    7211
2018-04    6939
2018-05    6873
2018-06    6167
2018-07    6292
2018-08    6512
2018-09      16
2018-10       4
Freq: M, Name: count, dtype: int64

In [9]:
# Convert the order purchase timestamp to datetime
df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'])

# Extract year-month from the order purchase date
df_orders['year_month'] = df_orders['order_purchase_timestamp'].dt.to_period('M')

# Count unique users per month
users_monthly = df_orders.groupby('year_month')['customer_id'].nunique()

In [10]:
users_monthly

year_month
2016-09       4
2016-10     324
2016-12       1
2017-01     800
2017-02    1780
2017-03    2682
2017-04    2404
2017-05    3700
2017-06    3245
2017-07    4026
2017-08    4331
2017-09    4285
2017-10    4631
2017-11    7544
2017-12    5673
2018-01    7269
2018-02    6728
2018-03    7211
2018-04    6939
2018-05    6873
2018-06    6167
2018-07    6292
2018-08    6512
2018-09      16
2018-10       4
Freq: M, Name: customer_id, dtype: int64